# Automated download of OpenStreetMap data 

In this notebook we will learn how to query a REST API using the `requests` package in Python. If it is not installed in your environment, install it using `conda install requests`. We will demonstrate it using the OHSOME API developed by HeiGIT and GIScience Research Group in Heidelberg.

In [1]:
import requests
import geopandas as gpd
%matplotlib inline
import mplleaflet

In [2]:
# this will only work if you have scikit-learn installed in your environment
# Create a memory cache so queries that have been sent before are not sent again
#from sklearn.externals.joblib import Memory
#memory = Memory(location='./tmp', verbose=0)

In [3]:
def _url(path):
    return 'https://api.ohsome.org/v0.9/' + path

## Query highways
Query all features with the tag landuse=grass

In [ ]:
keys = ["highway"]
values = [""]
bbox = "8.66744,49.41217,8.68465,49.42278"
types = ["WAY"]
date = "20180101"

In [ ]:
data = {'bboxes': bbox,
        'keys': ",".join(keys),
        'values': ",".join(values),
        'types': ",".join(types),
        'time': f'{date}',
        'format': 'geojson',
        'properties': 'tags',
        'showMetadata': 'false'}
response = requests.post(_url('elements/geometry'), data=data)

Check if the query returned a valid response:

In [ ]:
response.ok

Show the returned data in goejson format:

In [ ]:
response.json()

Load the data into a geopandas data frame:

In [ ]:
highways = gpd.GeoDataFrame().from_features(response.json())
highways.crs = {'init': 'epsg:4326'}
highways.head()

Let's buffer the highways:

In [ ]:
highways.plot()
mplleaflet.display()

## Query trees
Query all features with the tag natural=tree

In [ ]:
keys = ["natural"]
values = ["tree"]
bbox = "8.66744,49.41217,8.68465,49.42278"
types = ["NODE"]
date = "20180101"

In [ ]:
data = {'bboxes': bbox,
        'keys': ",".join(keys),
        'values': ",".join(values),
        'types': ",".join(types),
        'time': f'{date}',
        'format': 'geojson',
        'properties': 'tags',
        'showMetadata': 'false'}
response = requests.post(_url('elements/geometry'), data=data)

In [ ]:
trees = gpd.GeoDataFrame().from_features(response.json())
trees.crs = {'init': 'epsg:4326'}
trees = trees.to_crs({'init': 'epsg:32632'})
trees.head()

## Count the number of trees close to each highways segment

First, we will buffer the highways by 10 meters.

In [ ]:
highways_buffered = highways.to_crs({'init': 'epsg:32632'}).buffer(10)
highways_buffered.to_crs({'init': 'epsg:4326'}).plot()
mplleaflet.display()

Plot both trees and buffered highways

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 1, figsize=(10, 6))
trees.to_crs({'init': 'epsg:4326'}).plot(ax=axes, color="red")
highways_buffered.to_crs({'init': 'epsg:4326'}).plot(ax=axes)
mplleaflet.display()

Count the number of trees for each highway

In [ ]:
highways["green"] = highways_buffered.apply(lambda x: trees.within(x).sum())

In [ ]:
highways.plot(column="green", cmap="Greens")